In [58]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os

### Compilation of all appropriate CSV files

**More info to be added soon

In [59]:
#compilation of all appropriate csv files
main_path = os.getcwd() + "/CSV"

# Initializing dicts
balance_sheet_df_collection = {}
cashflow_statement_df_collection = {}
income_statement_df_collection = {}
market_stats_df_collection = {}

collections = {
    "balance_sheet_quarterly.csv": balance_sheet_df_collection,
    "cashflow_statement_quarterly.csv": cashflow_statement_df_collection,
    "income_statement_quarterly.csv": income_statement_df_collection,
    "market_stats.csv": market_stats_df_collection
}

def process_csv_file(current_path, folder):
    copy_df = pd.read_csv(current_path)
    copy_df["Unnamed: 0"] = (copy_df["Unnamed: 0"] >= 0).astype(int).replace(1, folder)
    column_names = copy_df.columns
    return pd.DataFrame(copy_df, columns=column_names)

for folder in os.listdir(main_path):
    for csv_file in os.listdir(os.path.join(main_path, folder)):
        current_path = os.path.join(main_path, folder, csv_file)
        
        if csv_file in collections:
            collections[csv_file][folder] = process_csv_file(current_path, folder)


balance_sheet_df = pd.concat(balance_sheet_df_collection)
cashflow_statement_df = pd.concat(cashflow_statement_df_collection)
income_statement_df = pd.concat(income_statement_df_collection)
market_stats_df = pd.concat(market_stats_df_collection)


### Clean up of compiled dataframes

 - balance_sheet_df
 - cashflow_statement_df
 - income_statement_df
 - market_stats_df

### Balance Sheet Data Cleanup

In [60]:

balance_sheet_df.rename(columns = {"Unnamed: 0":"Ticker", "Breakdown":"Date"}, inplace = True)
balance_sheet_df["Date"] = pd.to_datetime(balance_sheet_df["Date"])
balance_sheet_df.index.set_names(['Ticker Symbol', 'Index'], inplace = True) # Set index names 
# Clean column types
cols=[i for i in balance_sheet_df.columns if i not in ["Ticker","Date"]]
for col in cols:
    balance_sheet_df[col] = balance_sheet_df[col].astype(str).str.replace(',','')
    balance_sheet_df[col].replace('-', np.nan, inplace = True) #double check no negative values, find way to replace "-" for null values NaN without replacing negative values
    balance_sheet_df[col]= balance_sheet_df[col].astype(float)

In [68]:
balance_sheet_df["Total Assets"].dtypes

# Alternatively, you can use isinstance()
is_float = all(isinstance(value, float) for value in balance_sheet_df['Total Assets'])
print(is_float)  # Output: True

True


### Cashflow Statement Data Cleanup

In [62]:
cashflow_statement_df.rename(columns = {"Unnamed: 0":"Ticker", "Breakdown":"Date"}, inplace = True)
cashflow_statement_df.index.set_names(['Ticker Symbol', 'Index'], inplace = True) # Set index names 
cashflow_statement_df.drop(0, level=1, axis=0, inplace=True) # Erase all ttm values as not necessary (trailing twelve months data not necessary)
cashflow_statement_df["Date"] = pd.to_datetime(cashflow_statement_df["Date"]) # Updating column type to datetime

# Clean column types
cols=[i for i in cashflow_statement_df.columns if i not in ["Ticker","Date"]]
for col in cols:
    cashflow_statement_df[col] = cashflow_statement_df[col].astype(str).str.replace(',','')
    cashflow_statement_df[col].replace('-', np.nan, inplace = True) #double check no negative values, find way to replace "-" for null values NaN without replacing negative values but it works?
    cashflow_statement_df[col]= cashflow_statement_df[col].astype(float)

### Income Statement Data Cleanup

In [63]:
income_statement_df.rename(columns = {"Unnamed: 0":"Ticker", "Breakdown":"Date"}, inplace = True)
income_statement_df.index.set_names(['Ticker Symbol', 'Index'], inplace = True) # Set index names 
income_statement_df.drop(0, level=1, axis=0, inplace=True) # Erase all ttm values as not necessary (trailing twelve months data not necessary)
income_statement_df["Date"] = pd.to_datetime(income_statement_df["Date"]) # Updating column type to datetime

# Clean column types
cols=[i for i in income_statement_df.columns if i not in ["Ticker","Date"]]
for col in cols:
    income_statement_df[col] = income_statement_df[col].astype(str).str.replace(',','')
    income_statement_df[col].replace('-', np.nan, inplace = True) #double check no negative values, find way to replace "-" for null values NaN without replacing negative values but it works?
    income_statement_df[col]= income_statement_df[col].astype(float)

### Market Stats Data Cleanup

In [64]:
market_stats_df.rename(columns = {"Unnamed: 0":"Ticker", "Breakdown":"Date"}, inplace = True)
market_stats_df.index.set_names(['Ticker Symbol', 'Index'], inplace = True) # Set index names 
market_stats_df.drop(0, level=1, axis=0, inplace=True) # Erase all "as of date" rows (1st row for every ticker - unnecessary data)
market_stats_df["Date"] = pd.to_datetime(market_stats_df["Date"]) # Updating column type to datetime

# Clean column types (all numerical columns except market cap are floats)
cols=[i for i in market_stats_df.columns if i not in ["Ticker","Date"]]
for col in cols:
    market_stats_df[col].replace('-', np.nan, inplace = True) #double check no negative values, find way to replace "-" for null values NaN without replacing negative values but it works?

# replace "." with "" in only market cap and replace M with 4 zeros, replace B with 7 zeros
market_stats_df["Market Cap (intraday)"] = market_stats_df["Market Cap (intraday)"].astype(str).str.replace("B", "0000000")
market_stats_df["Market Cap (intraday)"] = market_stats_df["Market Cap (intraday)"].astype(str).str.replace("M", "0000")
market_stats_df["Market Cap (intraday)"] = market_stats_df["Market Cap (intraday)"].astype(str).str.replace(".", "")
market_stats_df["Market Cap (intraday)"] = market_stats_df["Market Cap (intraday)"].astype(str).astype(int)


### Filter dates for all files to Jan 2017 - March 2022

In [65]:
# Create a dictionary to store the DataFrames
dataframes = {
    'balance_sheet_df': balance_sheet_df,
    'cashflow_statement_df': cashflow_statement_df,
    'income_statement_df': income_statement_df,
    'market_stats_df': market_stats_df
}

# Define the cutoff date
cutoff_earliest_date = pd.to_datetime('2016-12-31')
cutoff_latest_date = pd.to_datetime('2022-04-01')

# Iterate over the dictionary items and filter the DataFrames
for df_name, df in dataframes.items():
    mask = (df['Date'] >= cutoff_earliest_date) & (df['Date'] <= cutoff_latest_date)
    dataframes[df_name] = df[mask]

# Access the updated DataFrames
balance_sheet_df = dataframes['balance_sheet_df'].reset_index().drop(columns=["Index", "Ticker"])
cashflow_statement_df = dataframes['cashflow_statement_df'].reset_index().drop(columns=["Index", "Ticker"])
income_statement_df = dataframes['income_statement_df'].reset_index().drop(columns=["Index", "Ticker"])
market_stats_df = dataframes['market_stats_df'].reset_index().drop(columns=["Index", "Ticker"])


### df to CSV

In [66]:
# Saves compiled dataframes as csv files under compiled_CSV
os.makedirs('compiled_CSV', exist_ok=True)  
balance_sheet_df.to_csv('compiled_CSV/balance_sheet.csv') 
cashflow_statement_df.to_csv('compiled_CSV/cashflow_statement.csv') 
income_statement_df.to_csv('compiled_CSV/income_statement.csv') 
market_stats_df.to_csv('compiled_CSV/market_stats.csv') 

### Loading objects/csv files to AWS S3

In [55]:
import boto3

access_key = os.environ.get('AWS_ACCESS_KEY_ID')
secret_key = os.environ.get('AWS_SECRET_ACCESS_KEY')

def get_file_paths(folder_path):
    file_paths = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.csv'):  # Filter CSV files
                file_path = os.path.join(root, file)
                file_paths.append(file_path)
    return file_paths

def upload_files_to_s3(file_paths, bucket_name):
    s3 = boto3.client('s3', aws_access_key_id=access_key, aws_secret_access_key=secret_key)
    for file_path in file_paths:
        file_name = file_path.split('/')[-1]  # Extract the file name from the path
        s3.upload_file(file_path, bucket_name, file_name)
        print(f"Uploaded {file_name} to {bucket_name}")

# Folder path containing the CSV files
folder_path = os.getcwd() + "/compiled_CSV"

# Call the function to get file paths
file_paths = get_file_paths(folder_path)

# Name of the S3 bucket
bucket_name = 'ds4a-c1-team22'

# Call the function to upload files
upload_files_to_s3(file_paths, bucket_name)

Uploaded balance_sheet.csv to ds4a-c1-team22
Uploaded market_stats.csv to ds4a-c1-team22
Uploaded company_info.csv to ds4a-c1-team22
Uploaded income_statement.csv to ds4a-c1-team22
Uploaded financial_ratios_df.csv to ds4a-c1-team22
Uploaded cashflow_statement.csv to ds4a-c1-team22


### Company Info CSV cleanup

In [56]:
market_stats_df.columns

Index(['Ticker Symbol', 'Date', 'Market Cap (intraday)', 'Enterprise Value',
       'Trailing P/E', 'Forward P/E', 'PEG Ratio (5 yr expected)',
       'Price/Sales', 'Price/Book', 'Enterprise Value/Revenue',
       'Enterprise Value/EBITDA'],
      dtype='object')

### Financial Ratio Calculations

In [57]:
balance_sheet_df.columns

Index(['Ticker Symbol', 'Date', 'Total Assets',
       'Cash, Cash Equivalents & Federal Funds Sold',
       'Cash And Cash Equivalents', 'Cash', 'Cash Equivalents',
       'Cash And Due from Banks', 'Interest Bearing Deposits Assets',
       'Restricted Cash And Investments',
       ...
       'Properties', 'Line of Credit', 'Unrealized Gain Loss',
       'Non Current Deferred Taxes Liabilities', 'Defined Pension Benefit',
       'Due to Related Parties', 'Current Deferred Liabilities',
       'Current Deferred Taxes Liabilities', 'Minimum Pension Liabilities',
       'Other Capital Stock'],
      dtype='object', length=133)

In [247]:
from dateutil.relativedelta import relativedelta
import datetime

financial_ratios = market_stats_df.drop(
  columns = [
    'Enterprise Value','Trailing P/E', 'Forward P/E', 
    'PEG Ratio (5 yr expected)','Price/Sales', 
    'Price/Book', 'Enterprise Value/Revenue', 'Enterprise Value/EBITDA'
    ]
  )

financial_ratios["ROE"] = (income_statement_df["Net Income Common Stockholders"] / balance_sheet_df["Common Stock Equity"]) * 100
financial_ratios["ROA"] = (income_statement_df["Net Income Common Stockholders"] / balance_sheet_df["Total Assets"]) * 100
#financial_ratios["CAR"] =

d1 = income_statement_df[["Date", "Ticker","Net Interest Income"]]
d2 = balance_sheet_df[["Date", "Ticker","Total Assets"]]


merge_NI_and_TA = pd.merge(d1, d2, on=['Date', 'Ticker'])
merge_NI_and_TA[merge_NI_and_TA["Date"] == merge_NI_and_TA["Date"] + pd.DateOffset(years=-1)]

# for index, row in merge_NI_and_TA.iterrows():
#     curr_year_total_assets = row["Total Assets"]
#     prior_year = pd.Timestamp(row["Date"].date() - relativedelta(years=1))
#     prior_year_total_assets = merge_NI_and_TA[merge_NI_and_TA["Date"] == prior_year]["Total Assets"]
#     # print(row["Net Interest Income"], curr_year_total_assets, prior_year_total_assets)
#     NI = row["Net Interest Income"]
#     val = NI / ((curr_year_total_assets + prior_year_total_assets) / 2)
#     print(merge_NI_and_TA.loc(merge_NI_and_TA["Average Total Assets"]))
#     print(merge_NI_and_TA.loc[ (merge_NI_and_TA["Date"] == row["Date"]) & (merge_NI_and_TA["Ticker"] == row["Ticker"]) ]["Average Total Assets"])


# financial_ratios["NIM"] = (income_statement_df["Net Interest Income"] / ((balance_sheet_df)/ 2)#missing average invested assets

#financial_ratios["NPL"] =

# financial_ratios

,Date,Ticker,Net Interest Income,Total Assets,Average Total Assets
0,2022-03-31,EWBC,415613.0,62241456.0,NaN
1,2021-12-31,EWBC,405697.0,60870701.0,0.0
2,2021-09-30,EWBC,395706.0,60959110.0,0.0
3,2021-06-30,EWBC,376473.0,59854876.0,NaN
4,2021-03-31,EWBC,353695.0,56874146.0,0.0
...,...,...,...,...,...
654,2017-12-31,SIVBQ,393706.0,51214467.0,0.0
655,2017-09-30,SIVBQ,373974.0,50754287.0,0.0
656,2017-06-30,SIVBQ,342696.0,48400379.0,0.0
657,2017-03-31,SIVBQ,309993.0,46413339.0,0.0
